<a href="https://colab.research.google.com/github/satyam26en/TIME_TABLE_KIIT/blob/main/MAIN_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [74]:
import pandas as pd

# Load the section, professional elective, and core section files from GitHub
section_url = 'https://raw.githubusercontent.com/satyam26en/TIME_TABLE_KIIT/main/SECTION.csv'
elective_url = 'https://raw.githubusercontent.com/satyam26en/TIME_TABLE_KIIT/main/Elective_TIME_TABLE.csv'
core_url = 'https://raw.githubusercontent.com/satyam26en/TIME_TABLE_KIIT/main/CORE_TIME_TABLE_2%20-%20Sheet1.csv'

section_df = pd.read_csv(section_url)
elective_df = pd.read_csv(elective_url)
core_df = pd.read_csv(core_url)

# Normalize the 'Roll No.' column to ensure there are no leading/trailing spaces and consistent data type
section_df['Roll No.'] = section_df['Roll No.'].astype(str).str.strip()

# Define the order of days and times in 12-hour format
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
times = ['8-9 AM', '9-10 AM', '10-11 AM', '11-12 PM', '12-1 PM', '1-2 PM', '2-3 PM', '3-4 PM', '4-5 PM']

# Create an empty DataFrame with days as columns and times as index
timetable_df = pd.DataFrame(index=times, columns=days)

# Function to take roll number input and display the timetable
def display_timetable(roll_number):
    # Find the section details for the given roll number
    student_section = section_df[section_df['Roll No.'] == roll_number]

    if student_section.empty:
        print("Roll number not found.")
        return

    # Extract the core section and elective sections
    core_section = student_section['Core Section'].values[0]
    elective_1_section = student_section['Professional Elective 1'].values[0]
    elective_2_section = student_section['Professional Elective 2'].values[0]

    # Retrieve the weekly timetable for Professional Electives 1 and 2
    elective_1_timetable = elective_df[elective_df['Section(DE)'] == elective_1_section]
    elective_2_timetable = elective_df[elective_df['Section(DE)'] == elective_2_section]
    core_timetable = core_df[core_df['Section'] == core_section]

    # Define the order of days and a mapping from abbreviated to full day names
    day_mapping = {
        'MON': 'Monday',
        'TUE': 'Tuesday',
        'WED': 'Wednesday',
        'THU': 'Thursday',
        'FRI': 'Friday',
        'SAT': 'Saturday'
    }

    # Convert time to 12-hour format
    time_mapping = {
        '8 TO 9': '8-9 AM',
        '9 TO 10': '9-10 AM',
        '10 TO 11': '10-11 AM',
        '11 TO 12': '11-12 PM',
        '12 TO 1': '12-1 PM',
        '1 TO 2': '1-2 PM',
        '2 TO 3': '2-3 PM',
        '3 TO 4': '3-4 PM',
        '4 TO 5': '4-5 PM'
    }

    # Function to format the timetable with subject names
    def format_timetable(timetable_df):
        timetable_dict = {day: {time: "x" for time in times} for day in days}
        room_columns = [col for col in timetable_df.columns if 'ROOM' in col]
        for index, row in timetable_df.iterrows():
            for col in room_columns:
                if row[col] != '---':
                    # Get time column name
                    time_col_index = timetable_df.columns.get_loc(col) + 1
                    time_col = timetable_df.columns[time_col_index]

                    # Get the day and time slot
                    day = row['DAY']
                    day_full = day_mapping.get(day, 'Unknown')
                    if day_full == 'Unknown':
                        continue  # Skip if day is not in the mapping

                    time_slot = time_mapping.get(time_col, time_col)

                    # Extract the subject name from the intersection of day and time slot
                    subject = row.get(time_col, 'N/A')
                    room_number = row[col]

                    if subject.lower() != 'x':  # Only include if it's not 'x'
                        timetable_dict[day_full][time_slot] = f"{subject} ({room_number})"
        return timetable_dict

    # Format the timetables
    elective_1_timetable_dict = format_timetable(elective_1_timetable)
    elective_2_timetable_dict = format_timetable(elective_2_timetable)
    core_timetable_dict = format_timetable(core_timetable)

    # Combine all timetables into a single dictionary
    combined_timetable_dict = {day: {time: "x" for time in times} for day in days}

    for day in days:
        for time_slot in times:
            if core_timetable_dict[day][time_slot] != "x":
                combined_timetable_dict[day][time_slot] = core_timetable_dict[day][time_slot]
            if elective_1_timetable_dict[day][time_slot] != "x":
                combined_timetable_dict[day][time_slot] = elective_1_timetable_dict[day][time_slot]
            if elective_2_timetable_dict[day][time_slot] != "x":
                combined_timetable_dict[day][time_slot] = elective_2_timetable_dict[day][time_slot]

    # Create a DataFrame from the combined timetable dictionary
    combined_timetable_df = pd.DataFrame(combined_timetable_dict)

    # Display the combined timetable
    print("\nCombined Timetable:")
    print(combined_timetable_df.to_string(index=True))

    # Save the DataFrame as a CSV file
    combined_timetable_df.to_csv('timetable.csv', index_label="Time")

# Example usage
roll_number = input("Enter Roll Number: ")
display_timetable(roll_number)


Enter Roll Number: 22052171

Combined Timetable:
                   Monday           Tuesday         Wednesday         Thursday            Friday        Saturday
8-9 AM                  x                 x    DAA (A-LH-107)       CN (B-402)                 x               x
9-10 AM                 x                 x                 x                x                 x               x
10-11 AM    EE (A-LH-107)                 x     SE (A-LH-107)  DAA(L) (WL-104)     EE (A-LH-207)               x
11-12 PM    SE (A-LH-004)                 x  DMDW(DE) (B-306)    SE (A-LH-005)                 x     SE (WL-103)
12-1 PM    DAA (A-LH-004)                 x   DOS(DE) (B-404)   DAA (A-LH-005)                 x  CN(L) (WL-103)
1-2 PM      CN (A-LH-004)                 x                 x    EE (A-LH-005)  DMDW(DE) (B-306)  CN(L) (WL-103)
2-3 PM                  x                 x                 x                x                 x               x
3-4 PM    DOS(DE) (B-404)                 x    